In [ ]:
import pandas as pd

In [ ]:
# orders.csv
url = "https://drive.google.com/file/d/1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders = pd.read_csv(path)

# orderlines.csv
url = "https://drive.google.com/file/d/1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines = pd.read_csv(path)

# products.csv
url = "https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)

# brands.csv
url = "https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
brands = pd.read_csv(path)

In [ ]:
orders_df = orders.copy()
orderlines_df = orderlines.copy()
products_df = products.copy()
brands_df = brands.copy()

#Orderlines cleaning
- no duplication to remove
- no missing value to remove
- convert date to datetime type
- convert unit_price to float type by shifting the 2nd decimal up one position



**Duplications**: there is NO duplication to be removed

In [ ]:
orderlines_df.duplicated().sum()

0

**Missing Values**: there is NO missing value to be removed

In [ ]:
orderlines_df.isna().sum()

,0
id,0
id_order,0
product_id,0
product_quantity,0
sku,0
unit_price,0
date,0


**Wrong Data Types**: 'date' should be datetime type and 'unit_price' should be float type

In [ ]:
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


convert 'date' to datetime type

In [ ]:
orderlines_df['date'] = pd.to_datetime(orderlines_df['date'])

In [ ]:
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  object        
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 15.7+ MB


convert 'unit_price' to float using a method to remove the first decinal AND shift the second decimal up for one position ex. 1.004.99 to 100.50

In [ ]:
#to show that simply converting 'unit_price' to float is not possible due to the wrong format
#orderlines_df['unit_price'] = pd.to_numeric(orderlines_df['unit_price'])

In [ ]:
# create a copy of orderlines with all faulty prices as x.xxx.xx
fault_orderlines = orderlines_df.loc[orderlines_df['unit_price'].str.match(r'\d{1,3}\.\d{3}\.\d{2}') == 1].copy() # also for better understanding: match(r'\d*\.\d*\.\d*') does the same \d*=any number of digits from 0 to ...
fault_orderlines = fault_orderlines.reset_index()

In [ ]:
index = 0
for i in fault_orderlines['unit_price']:   # i loops through all unit_prices
  i = i.replace(".", "")                   # deletes all '.'
  i = i[:-3]+"."+i[-3:]                    # first everything before + '.'  last three numbers
  fault_orderlines.loc[index, 'unit_price'] = i    # overwrite into unit price
  index += 1

In [ ]:
new_unit_price = fault_orderlines[['index', 'unit_price']].set_index('index')

In [ ]:
orderlines_cl = orderlines_df.copy()
orderlines_cl.update(new_unit_price)            # update new unit price into whole orderlines

In [ ]:
orderlines_cl['unit_price'] = pd.to_numeric(orderlines_cl['unit_price']).round(2)   # convert to float and round it to two decimals
orderlines_cl

,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38
...,...,...,...,...,...,...,...
293978,1650199,527398,0,1,JBL0122,42.99,2018-03-14 13:57:25
293979,1650200,527399,0,1,PAC0653,141.58,2018-03-14 13:57:34
293980,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41
293981,1650202,527388,0,1,BEZ0204,19.99,2018-03-14 13:58:01


In [ ]:
#now the unit_price is float
orderlines_cl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  float64       
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 15.7+ MB


#Products Cleaning
-  8746 duplications to remove
-  missing values in price and type to remove (not in desc)
-  convert price to float datatype by removing the wrong format
-  ignore the promo_price as it is not meaningful (can remove column later if needed)
-  the result file has not sku duplication to remove


**Duplications**: there are 8746 duplicates to be removed

In [ ]:
products_df.head(5)

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364


In [ ]:
products_df.shape[0]

19326

In [ ]:
products_df.duplicated().sum()

8746

In [ ]:
products_df = products_df.drop_duplicates()

In [ ]:
products_df.shape[0]

10580

**Missing Values**: there are 46 missing prices and 50 missing types to be removed

In [ ]:
products_df.isna().sum()

,0
sku,0
name,0
desc,7
price,46
promo_price,0
in_stock,0
type,50


In [ ]:
# NaNs in products
#entry without 'desc' is fine, so there is no need to remove it
#entry without 'price' needs to be removed
#enrty without 'type' can be investigated to save, however, the number is so low it is not worth it so simply delete

na_price = products_df['price'].isna() == 0    # 0 == False
na_type = products_df['type'].isna() == 0
products_df = products_df.loc[na_price & na_type, :]

In [ ]:
products_df.isna().sum()

,0
sku,0
name,0
desc,7
price,0
promo_price,0
in_stock,0
type,0


**Wrong Data Type**: 'price' and 'promo_price' should be float
However, there are both in the wrong format of 1.004.99 so cannot simeply be converted
price: number is very low, so their rows can simply be all removed
promo_price: such a high number, but as the column is not so meaningful, we can ignore it or remove the column

In [ ]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10484 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10484 non-null  object
 1   name         10484 non-null  object
 2   desc         10477 non-null  object
 3   price        10484 non-null  object
 4   promo_price  10484 non-null  object
 5   in_stock     10484 non-null  int64 
 6   type         10484 non-null  object
dtypes: int64(1), object(6)
memory usage: 655.2+ KB


In [ ]:
products_df.shape[0]

10484

In [ ]:
# Dana Code
# faulty prices
fault_products = products_df.loc[products_df['price'].str.match(r'\d*\.\d*\.\d*') == 1].copy() # also for better understanding: match(r'\d*\.\d*\.\d*') does the same \d*=any number of digits from 0 to ...
fault_products = fault_products.reset_index()
fault_products

,index,sku,name,desc,price,promo_price,in_stock,type
0,665,CRU0015-2,Crucial memory Mac 16GB (2x8GB) SO-DIMM DDR3 1...,RAM 16GB (2x8GB) 135V MacBook Pro iMac (2012/2...,1.639.792,1.629.894,1,1364
1,827,PAC0339,NewerTech miniStack 4TB Hard Drive Mac,External Box Hard Drive Mac + 4TB.,2.199.791,2.199.901,0,11935397
2,885,PAC0376,OWC Mercury Elite Pro Dual Thunderbolt + 8TB,RAID outer box 35 inch SATA connection Thunder...,5.609.698,5.549.895,0,11935397
3,898,REP0156,iPhone 5 GSM antenna repair,Repair service including parts and labor for i...,69.989.909,699.899,0,"1,44E+11"
4,941,REP0185,Home button repair iPad mini,Repair service including parts and labor for i...,69.989.909,699.899,0,"1,44E+11"
...,...,...,...,...,...,...,...,...
369,19312,REP0424,Input repair Headphones iPad,Repair service including parts and labor for iPad,6.999.003,69.99,0,"1,44E+11"
370,19313,REP0421,iPad charging connector repair,Repair service including parts and labor for iPad,6.999.003,69.99,0,"1,44E+11"
371,19314,REP0416,iPad front camera repair,Repair service including parts and labor for iPad,6.999.003,69.99,0,"1,44E+11"
372,19315,REP0413,repair rear camera iPad,Repair service including parts and labor for iPad,6.999.003,69.99,0,"1,44E+11"


In [ ]:
# new version with pd numeric
fault_products['price'] = fault_products['price'].str.replace(".", "",1)    # delete only first '.'
fault_products['price'] = pd.to_numeric(fault_products['price'])
fault_products['price'] = fault_products['price']/10                        # shift by one decimal


# values about 10.000 need to be shifted another decimal
mask = fault_products['price'] > 10000
fault_products.loc[mask,'price'] = fault_products.loc[mask ,'price']/10

In [ ]:
# still some faulty things with this type
mask = fault_products['type'] != '1,44E+11'
fault_products = fault_products.loc[mask,:]

In [ ]:
fault_products.shape

(299, 8)

In [ ]:
fault_products = fault_products.set_index('index')
fault_products

,sku,name,desc,price,promo_price,in_stock,type
index,,,,,,,
665,CRU0015-2,Crucial memory Mac 16GB (2x8GB) SO-DIMM DDR3 1...,RAM 16GB (2x8GB) 135V MacBook Pro iMac (2012/2...,163.9792,1.629.894,1,1364
827,PAC0339,NewerTech miniStack 4TB Hard Drive Mac,External Box Hard Drive Mac + 4TB.,219.9791,2.199.901,0,11935397
885,PAC0376,OWC Mercury Elite Pro Dual Thunderbolt + 8TB,RAID outer box 35 inch SATA connection Thunder...,560.9698,5.549.895,0,11935397
1057,MOP0057,Mophie Space Pack Battery Case (1700mAh) and S...,Housing with battery and 16GB external storage...,132.9911,1.329.911,0,"5,49E+11"
1058,MOP0058,Mophie Space Pack Battery Case (1700mAh) and S...,Housing with battery and 32GB external storage...,159.9862,1.599.862,0,"5,49E+11"
...,...,...,...,...,...,...,...
19248,DJI0026,DJI Mavic Air Drone cuadricóptero Arctic White,Drone cuadricóptero laptop with integrated cam...,8490.0013,8.490.001,0,11905404
19249,DJI0025,DJI Mavic Air Drone Black Onyx cuadricóptero,Drone cuadricóptero laptop with integrated cam...,8490.0013,8.490.001,0,11905404
19250,LIN0013,Linksys Wi-Fi Velop system AC2200 1 unit,Wi-Fi high-density intelligent Mesh technology,1999.9001,199.99,0,1334


In [ ]:
products_cl = products_df.copy()
products_cl['price'] = pd.to_numeric(products_cl['price'],errors='coerce')    # converts to float and coerce set every xx.xxx.xx to NaN
#products_cl.dropna(axis=0) 	# drops whole row_0, col_1 with NaN, not updating

In [ ]:
mask = products_cl['price'].isna() # true for NaN
# delete all NaN (rows)
products_cl = products_cl.loc[~mask,:]    # products table without xx.xxx.xx

In [ ]:
# concatening
products_cl = pd.concat([products_cl,fault_products], axis=0)

In [ ]:
products_cl

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.9900,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59.0000,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59.0000,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25.0000,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.9900,31.99,1,1364
...,...,...,...,...,...,...,...
19248,DJI0026,DJI Mavic Air Drone cuadricóptero Arctic White,Drone cuadricóptero laptop with integrated cam...,8490.0013,8.490.001,0,11905404
19249,DJI0025,DJI Mavic Air Drone Black Onyx cuadricóptero,Drone cuadricóptero laptop with integrated cam...,8490.0013,8.490.001,0,11905404
19250,LIN0013,Linksys Wi-Fi Velop system AC2200 1 unit,Wi-Fi high-density intelligent Mesh technology,1999.9001,199.99,0,1334
19251,LIN0014,Linksys Wi-Fi Velop system AC4400 2 units,Wi-Fi high-density intelligent Mesh technology,299.9905,2.999.905,1,1334


In [ ]:
products_cl.isna().sum()

,0
sku,0
name,0
desc,7
price,0
promo_price,0
in_stock,0
type,0


In [ ]:
# delte promo columun
products_cl = products_cl.drop('promo_price',axis=1)

In [ ]:
#now the 'price' column is float
products_cl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10409 entries, 0 to 19252
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sku       10409 non-null  object 
 1   name      10409 non-null  object 
 2   desc      10402 non-null  object 
 3   price     10409 non-null  float64
 4   in_stock  10409 non-null  int64  
 5   type      10409 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 569.2+ KB


**sku duplicated cleaning**
There is no duplicated sku in this state to be removed

In [ ]:
products_cl['sku'].duplicated().sum()

0

#Orders cleaning
-  no duplication to remove
-  5 missing values in total_paid to remove
-  convert created_date to datetime type

**Duplications**: there is no dupliacte to be removed

In [ ]:
orders_df.duplicated().sum()

0

**Missing Values**: only 5 missing 'tota_paid' which all are in pending state to be removed

In [ ]:
orders_df.isna().sum()

,0
order_id,0
created_date,0
total_paid,5
state,0


In [ ]:
orders_df.loc[orders_df['total_paid'].isna() == 1, :]

,order_id,created_date,total_paid,state
127701,427314,2017-11-20 18:54:39,NaN,Pending
132013,431655,2017-11-22 12:15:24,NaN,Pending
147316,447411,2017-11-27 10:32:37,NaN,Pending
148833,448966,2017-11-27 18:54:15,NaN,Pending
149434,449596,2017-11-27 21:52:08,NaN,Pending


In [ ]:
na_total = orders_df['total_paid'].isna() == 1
orders_df = orders_df.loc[~na_total, :]

In [ ]:
orders_df.isna().sum()

,0
order_id,0
created_date,0
total_paid,0
state,0


**Wrong Data Type**: 'created_date' should be datetime datatype

In [ ]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226904 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226904 non-null  int64  
 1   created_date  226904 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226904 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 8.7+ MB


In [ ]:
#convert 'created_date' to Datetime data type
orders_df['created_date'] = pd.to_datetime(orders_df['created_date'])

In [ ]:
orders_cl = orders_df.copy()
#'created_date' is now a datetime type
orders_cl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226904 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      226904 non-null  int64         
 1   created_date  226904 non-null  datetime64[ns]
 2   total_paid    226904 non-null  float64       
 3   state         226904 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 8.7+ MB


#Brands

# Quality Cleaning

**orders_cl & orderlines_cl**:
-  keep only Completed states in orders_cl
-  remove non-existance order_id in both tables


In [ ]:
#exclue incompleted states
state_mask = orders_cl['state'].isin(['Pending', 'Shopping Basket', 'Cancelled', 'Place Order'])    # completed is taken only
orders_state = orders_cl.loc[~state_mask, :].copy()

#exclude unmatched order id
completed_order_ids = orders_state['order_id'].unique()

orderlines_completed = orderlines_cl.loc[(orderlines_cl['id_order'].isin(completed_order_ids)), :]
orders_completed = orders_state.loc[(orders_state['order_id'].isin(orderlines_completed['id_order'])), :]

**orderlines_completed & products_cl:**
-  keep only orderlines which have sku info in the products_cl table

In [ ]:
# 1. Found corrupted orderlines
known_products = products_cl['sku'].unique()
orderlines_completed.loc[:, 'Corrupted'] = False
orderlines_completed.loc[~orderlines_completed['sku'].isin(known_products), 'Corrupted'] = True

# 2. Found orders with corrupted orderlines
corrupted_order_ids = orderlines_completed.loc[orderlines_completed['Corrupted']==True, 'id_order'].unique()

# 3. Drop those orders
orders_qu = orders_completed.loc[~orders_completed['order_id'].isin(corrupted_order_ids)].copy()

# 4. Drop all orderlines from corrupted orders
orderlines_qu = orderlines_completed.loc[~orderlines_completed['id_order'].isin(corrupted_order_ids)].copy()

<ipython-input-48-59cd37559cc0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orderlines_completed.loc[:, 'Corrupted'] = False


# Final files
-  orders_qu
-  orderlines_qu
-  products_cl

In [ ]:
# save it later to a csv file
from google.colab import drive
drive.mount('/drive')

orders_qu.to_csv('/drive/My Drive/Eniac_promo/orders_qu.csv', index='False')
orderlines_qu.to_csv('/drive/My Drive/Eniac_promo/orderlines_qu.csv', index='False')
products_cl.to_csv('/drive/My Drive/Eniac_promo/products_cl.csv', index='False')
brands.to_csv('/drive/My Drive/Eniac_promo/brands.csv', index='False')

Mounted at /drive
